In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torchsummary import summary

filters = 96
blocks = 16

class ResBlock(nn.Module):
    def __init__(self, filters=filters, kernel_size=3):
        super(ResBlock, self).__init__()
        self.relu = nn.ReLU()
        self.conv0 = nn.Conv2d(filters, filters, kernel_size=kernel_size, padding='same')
        self.conv1 = nn.Conv2d(filters, filters, kernel_size=kernel_size, padding='same')
        self.conv2 = nn.Conv2d(filters, filters, kernel_size=kernel_size, padding='same')

    def forward(self, input):
        x = self.relu(self.conv0(input))
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x + input

class Model(nn.Module):
    def __init__(self, filters=filters, kernel_size=3, upscale_factor=2):
        super(Model, self).__init__()
        self.conv0 = nn.Conv2d(1, filters, kernel_size=kernel_size, padding='same')
        self.res_blocks = nn.ModuleList([ResBlock() for _ in range(blocks)])
        self.conv1 = nn.Conv2d(filters, filters, kernel_size=kernel_size, padding='same')
        self.feats_conv = nn.Conv2d(filters, 4, kernel_size=kernel_size, padding='same')
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

    def forward(self, input):
        conv0 = self.conv0(input)
        x = conv0
        for block in self.res_blocks:
            x = block(x)
        conv1 = self.conv1(x)
        features = self.feats_conv(conv1 + conv0)
        outputs = torch.clip(self.pixel_shuffle(features), 0.0, 1.0)
        return outputs

model = Model()
model.load_state_dict(torch.load("/content/r8f64_torch.pth"))
summary(model.cuda(), (1, 256, 256))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/Datasets/Anime_Train_HR.zip /content/HR1.zip
!cp /content/drive/MyDrive/Datasets/Digital_Art_Train_HR.zip /content/HR2.zip
!unzip /content/HR1.zip
!unzip /content/HR2.zip

In [ ]:
# Single Dataset Gray
import glob
import cv2
import numpy as np

filelist = sorted(glob.glob('/content/HR/*.png'))
train_ref = []
train_in = []

for myFile in filelist:
    image = cv2.imread(myFile, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0)
    train_ref.append(image)
    image = cv2.resize(image, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR_EXACT)
    train_in.append(image)

train_ref = np.array(train_ref).astype(np.float32) / 255.0
train_ref = np.clip(train_ref, 0.0, 1.0)
train_ref = np.expand_dims(train_ref, axis=-1)
train_ref = np.transpose(train_ref, (0, 3, 1, 2))
print(train_ref.shape)

train_in = np.array(train_in).astype(np.float32) / 255.0
train_in = np.clip(train_in, 0.0, 1.0)
train_in = np.expand_dims(train_in, axis=-1)
train_in = np.transpose(train_in, (0, 3, 1, 2))
print(train_in.shape)

# Convert data to PyTorch tensors
train_in_tensor = torch.tensor(train_in)
train_ref_tensor = torch.tensor(train_ref)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms.functional as TF

# Create DataLoader for training data
train_dataset = TensorDataset(train_in_tensor, train_ref_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001)

# Move model to device if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()  # Set the model to train mode
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)

    # Print epoch statistics
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.8f}')

print('Finished Training')
torch.save(model.state_dict(), "/content/r8f64_torch.pth")
!mv /content/r8f64_torch.pth /content/drive/MyDrive/tmp/r8f64_torch.pth

In [ ]:
!mv /content/r8f64_torch.pth /content/drive/MyDrive/tmp/r8f64_torch.pth

In [ ]:
import torch
import cv2
import numpy as np

image = cv2.imread('/content/downscaled.png', cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0)
image = np.array(image).astype(np.float32) / 255.0
image = np.expand_dims(image, axis=-1)
image = np.expand_dims(image, axis=0)
image = np.transpose(image, (0, 3, 1, 2))
image = torch.tensor(image)

model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image = image.to(device)

with torch.no_grad():
    output = model(image)

output = output.cpu().numpy()
output = np.squeeze(output)
output = np.clip(output, 0.0, 1.0)
output = np.around(output * 255.0)
output = output.astype(np.uint8)

cv2.imwrite('/content/prediction.png', output)

In [ ]:
!pip install onnx
import onnx

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

x = torch.ones((1, 1, 1920, 1080))  # N x C x W x H
x = x.to(device)
torch.onnx.export(
    model, x, '/content/r8f64.onnx',
    input_names = ['input'],
    output_names = ['output'],
    dynamic_axes={
        'input' : {0 : 'batch', 2: 'width', 3: 'height'},
        'output' : {0 : 'batch', 2: 'width', 3: 'height'},
    }
)